Must ensure that the labels used are the cleaned ones after check_results

In [1]:
from transformers import pipeline
import json

c:\Users\Zade\Desktop\PythonStuff\jobReqs\jobReqs\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [3]:
with open(fr"data/p-raw_data-22-10-23.json") as f:
    data = json.load(f)

In [4]:
data['0aa33d3c84db9848']['techs']

['sap bi', 'sap basis', 'sap bi roles']

In [5]:
for key in data:
    if key.startswith('meta'):
        continue
    else:
        if len(data[key]['techs']) > 5:
            print(key, data[key]['techs'])

0ff5fd9855b6a27d ['analytical thinking', 'communication skills', 'curiosity', 'prototype culture', 'problem solving', 'collaboration', 'technical solutions', 'templates', 'data integration', 'data science', 'data analytics', 'data engineering', 'data visualization', 'best practices', 'data sources', 'feature and model evaluation', 'data transformation', 'data cleansing', 'data organization', 'data standardization', 'machine learning', 'neural networks', 'data visualization', 'statistical exploratory data analysis', 'analytics programming languages', 'frameworks', 'visualization tools', 'complex data analysis', 'time management', 'relational databases', 'advanced sql techniques', 'feature discovery', 'statistical modeling', 'machine learning algorithms', 'natural language processing technologies', 'deep learning technologies', 'python data analysis frameworks', 'data visualization tools', 'imbalanced data', 'git version control', 'automl tools', 'nosql databases', 'graph databases', 'js

In [6]:
test_jd = data['b6322345d39699fd']['cleaned_desc']

In [7]:
test_jd

" Perform testing and quality control on analytic models to ensure a smooth production release using Python, R, Scala, etc.  \n Conducting research, requirements design and validation of new Analytic products.  \n Properly document code, artifact description, implementation, and use cases  \n Provides data analytic support to client and leadership teams with reports, analytics, and with other ad hoc requests.  \n Understand the current reporting process and work out ways to optimize and enhance regular reports.  \n Coordinate with different functional teams to help implement models and monitor outcomes  \n Assists modelers with data-related questions to ensure models run efficiently and use the correct data  \n Triages, investigates, and helps resolve data issues  \n Customer and end user support and communication  \n Creation of training videos for the Analytic platform  \n \n \n Qualifications \n \n \n Required Qualifications:    \n \n Bachelor's degree with 0-2 years of experience O

Word based tokenization:
Similar words have different meanings.  
Vocab can end up large, very heavy model - large # weights.  can ignore words to limit amount.  e.g. take top 10,000 most common words could be done.  OOV words are lost.
Not great for me as we want it to learn previously unknown techs.

Character based tokenization:
num. characters is fairly low, maybe around 256 for all letters, numbers, special characters in english.  chinese could get up to 20,000 but we will focus on english
OOV tokens are less frequent, can tokenize misspelt words.  
but characters hold less info than words themselves in english.  must use lots of tokens at once, lowers the amount of context that can be used at once.
Can still be decent, not like I need a LOT of context for mine.

Subword based tokenization:
Middle ground between word and character based.  
Freq. words should not be split int osmaller subwords.
rare words should be decomposed into meaningful subwords

Other techs exist.  e.g. BERT uses WordPiece

Tokenizer pipeline


Raw text --> tokens --> special tokens (e.g. start of sentence, etc) --> input IDs

from transformers import AutoTokenizer

tokenizer = autoTokenizer.from_pretrained("model to use")
tokens = tokenizer.tokenize("text example")
input_ids = tokenizer.convert_tokens_to_ids(tokens) 

is the same as

tokenizer = AutoTokenizer.from_pretrained("model to use")
inputs = tokenizer("text example")

decode them with tokenizer.decode(inputs) # where inputs is something like [635, 37586, 2342, ...], our tokenized text

Batch inputs together

Inputs will not usually have the exact same length.  Must make this so for the tensors as they must all be rectangular.  Do this by padding and truncating.  Truncating is only done when sentence length is longer than max length allowed by model, as otherwise you lose info.

Models are pretrained with a specific character by model.  Find with:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token_id # this is the character used by the model to pad.

Attention layers look at padding tokens.  To fix this, tell attention layer to ignore tokens marked with the pad_token_id.  Do this via an attention mask.  
All done behind the scenes by the tokenizer with padding=True

For facebook/bart-large-mnli the max model length is 1024.

##For batch tokenization, can pad to the max batch size rather than the max length in the entire dataset saving a lot of time/cost!
This is called dynamic padding. Do this via a  DataCollatorWithPadding(tokenizer)
Note this may not work on all accelerators.  Never works on a TPU.

Can do a lot of this together with the .to_tf_dataset() method.

Fine tuning with keras best with a GPU.

In [8]:
from transformers import AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

In [10]:
tokenizer.model_max_length

1024